In [1]:
%%capture
%load_ext autoreload
%autoreload 2

# We can also put these magic commands in `.vscode/settings.json` like this:
# "jupyter.runStartupCommands": [
#     "%load_ext autoreload",
#     "%autoreload 2"
# ]

In [35]:
from pathlib import Path

import polars as pl

from cmipiu.features import aggregate_pq_files

print("All imports loaded successfully")

All imports loaded successfully


In [39]:
DATA_DIR = Path("../input/child-mind-institute-problematic-internet-use/")
files = [file for file in (DATA_DIR / 'series_train.parquet').iterdir()]

cols = ["X", "Y", "Z", "enmo", "light", "battery_voltage"]
train_agg = aggregate_pq_files(files, cols)
train_agg.head()

Aggregating pq files: 100%|██████████| 996/996 [00:13<00:00, 72.52it/s]


id,X_min,Y_min,Z_min,enmo_min,light_min,battery_voltage_min,X_mean,Y_mean,Z_mean,enmo_mean,light_mean,battery_voltage_mean,X_max,Y_max,Z_max,enmo_max,light_max,battery_voltage_max,X_std,Y_std,Z_std,enmo_std,light_std,battery_voltage_std
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""6b6467f4""",-0.992422,-3.549023,-1.008633,0.0,0.0,4086.0,-0.42547,0.056812,-0.307925,0.053125,10.545769,4095.827393,1.006836,1.009336,0.967422,3.42929,266.0,4145.0,0.471329,0.513878,0.393905,0.155176,15.383665,6.687519
"""9d6b1410""",-1.204403,-3.650786,-1.004756,0.0,0.0,3098.166748,0.20751,-0.019417,0.053819,0.060859,121.138039,3840.148926,1.796815,2.623967,3.006071,4.794266,2636.75,4158.0,0.562766,0.493536,0.593948,0.177751,379.6315,167.240082
"""0d01bbf2""",-3.29879,-3.262288,-1.134395,0.0,0.0,3706.0,-0.478971,-0.037643,-0.215953,0.06154,41.466927,3876.09668,1.159667,2.525316,1.802745,4.568309,2659.666748,4179.0,0.429476,0.518888,0.422696,0.129369,180.046051,121.027328
"""19455336""",-3.150714,-4.179972,-1.019038,0.0,0.0,3098.166748,-0.067279,0.187804,-0.381716,0.01887,10.622702,3828.518066,2.427422,2.343212,2.094606,5.087605,2408.199951,4133.0,0.282437,0.528473,0.669284,0.075737,39.840015,147.438263
"""2812951b""",-2.683229,-3.152597,-1.078669,0.0,0.0,3267.0,0.018125,0.001338,0.023644,0.086539,101.366875,3966.00708,2.936888,3.149463,1.820917,5.086066,2650.399902,4199.0,0.622624,0.566363,0.488903,0.183112,361.444977,153.062592


In [38]:
files = [file for file in (DATA_DIR / 'series_test.parquet').iterdir()]

cols = ["X", "Y", "Z", "enmo", "light", "battery_voltage"]
test_agg = aggregate_pq_files(files, cols)
test_agg.head()

Aggregating pq files: 100%|██████████| 2/2 [00:00<00:00, 50.26it/s]


id,X_min,Y_min,Z_min,enmo_min,light_min,battery_voltage_min,X_mean,Y_mean,Z_mean,enmo_mean,light_mean,battery_voltage_mean,X_max,Y_max,Z_max,enmo_max,light_max,battery_voltage_max,X_std,Y_std,Z_std,enmo_std,light_std,battery_voltage_std
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""00115b9f""",-1.746094,-2.905339,-1.048372,0.0,0.0,3824.0,-0.316383,0.016009,-0.167891,0.047388,42.296921,4053.241211,1.507865,1.666354,1.546979,4.004276,2633.25,4188.5,0.453665,0.502702,0.58571,0.106351,208.168976,112.404037
"""001f3379""",-1.038711,-1.52269,-1.018787,0.0,0.0,3098.166748,-0.004272,0.016859,-0.631096,0.011926,16.770199,3834.537598,1.034351,1.946303,1.146284,2.952888,2597.800049,4175.0,0.351545,0.303812,0.623476,0.024331,95.327438,155.573868


In [41]:
train = pl.read_csv(DATA_DIR / "train.csv")
test = pl.read_csv(DATA_DIR / "test.csv")

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")

Train shape: (3960, 82)
Test shape: (20, 59)


In [44]:
train = train.join(train_agg, how='left', on='id')
test = test.join(test_agg, how='left', on='id')

print(f"New train shape after joining aggregates: {train.shape}")
print(f"New test shape after joining aggregates: {test.shape}")

New train shape after joining aggregates: (3960, 106)
New test shape after joining aggregates: (20, 83)


In [47]:
train_filepath = Path('../input/processed/train_combined.csv')
train_filepath.parent.mkdir(parents=True, exist_ok=True)
train.write_csv(train_filepath)
print(f"Preprocessed training data saved in {train_filepath}")

test_filepath = Path('../input/processed/test_combined.csv')
test_filepath.parent.mkdir(parents=True, exist_ok=True)
test.write_csv(test_filepath)
print(f"Preprocessed testing data saved in {test_filepath}")

Preprocessed training data saved in ../input/processed/train_combined.csv
Preprocessed testing data saved in ../input/processed/test_combined.csv
